# Climate Cloud

### Data Processing

The following function converts a CDF file into a csv keeping only the values in our specified lat, lon limits as well as dropping unecessary data columns 

In [ ]:
import xarray as xr
# single file
nc_file = ""
csv_file = ""
dataDIR = 'nc_file'
data = xr.open_dataset(dataDIR)
df = data.to_dataframe()
df = df.drop('Lambert_Conformal', axis=1)
df = df.drop('time_bnds', axis=1)
index_drop = df[ (df['lat'] < 32.5350) | (df['lat'] >42.0056) ].index
df.drop(index_drop, inplace=True)
index_drop = df[ (df['lon'] < -124.2126) | (df['lon'] > -114.1312)].index 
df.drop(index_drop, inplace=True)
print(df)
df.to_csv(csv_file, index=True)
#df = df.drop('Lambert_Conformal', axis=1)

data folder clean up

In [ ]:
import os

directory = "data/2021"#change for each year 
old_format = "usdm_%Y%m%d.json"
new_format = "%Y%m%d.json"

for filename in os.listdir(directory):
    if filename.startswith("usdm_") and filename.endswith(".json"):
        old_path = os.path.join(directory, filename)
        new_path = os.path.join(directory, filename.replace("usdm_", "").replace("-", ""))
        os.rename(old_path, new_path)
        print(f"Renamed {old_path} to {new_path}")

The following functions are involved in combining the NOAA data with the USDM data

In [ ]:
import csv
import json
import re
import os

from shapely.geometry import shape, Point

#change based on year dealing with
directory = '2015drought_json'
directory_path = '2015drought_json'
csv_path ='cdf/apcp.2015.csv'
csv_out ='2015_final.csv'
#filelist = sorted(os.listdir(directory))

the function date_change changes the a given date in the format 2021-01-01 to 20210101. This allows for easier searching and file matching

In [ ]:
def date_change(date):
    date_str = date
    date_str_without_dashes = date_str.replace("-", "")
    return date_str_without_dashes

given the coordinates and a file name, catchecker checks to see if the point lies in a polygon. If it does it returns the corresponding drought catagory. Else a 0 is returned 

In [ ]:
def catchecker(lon,lat,filename):
    path = os.path.join(directory, filename)
    with open(path) as f:
        js = json.load(f)
    point = Point(lon, lat)#SCU coord
    for feature in js['features']:
        polygon = shape(feature['geometry'])
        if polygon.contains(point):
            return feature.get("id")
        else:
            return 0

This function returns the first date and last date in the folder for geojson data

In [ ]:
def min_max_date(flag,directory):

    list = sorted(os.listdir(directory))
    if(flag == "min"):
        #print("min_date")
        #print(list[0])
        return list[0]
    elif(flag == "max"):
       # print("max_date")
        #print(list[-1])
        return list[-1]
    else:
        print("error flag must be min/max")

In [ ]:

def folder_toset(directory_path):
  files = os.listdir(directory_path)
  json_files = [f.replace(".json", "") for f in files]
  set(json_files)
  return json_files


This function takes in directory ,the date,mindate & the max date to return the appropriate json file 

In [ ]:

#takes in directory ,the date,mindate & the max date to return the appropriate json file 
def date_match(dir_list,search_string,min_date,max_date):
  #print(dir_list)
  if search_string in dir_list:
        #append
    print("correct geo json found ")
    json_file_name = search_string + ".json"
    print(json_file_name)
    return json_file_name
  elif(search_string<min_date):
    print("date less than min date")
    mod_date= int(search_string)
    while(mod_date not in dir_list):
      int_mod_date = int(mod_date)
      int_mod_date+=1
      mod_date = str(int_mod_date)
    print(mod_date)
    converted_date=str(mod_date)
    json_file_name = converted_date + ".json"
    print(json_file_name)
    return json_file_name
  elif(search_string>max_date):
    print("date greater than max date")
    mod_date= int(search_string)
    while(mod_date not in dir_list):
      int_mod_date = int(mod_date)
      int_mod_date-=1
      mod_date = str(int_mod_date)
    print(mod_date)
    converted_date=str(mod_date)
    json_file_name = converted_date + ".json"
    print(json_file_name)
    return json_file_name
  else:
    print("date is in between sets")
    #if in between we want to use the data of the week ahead 
    mod_date= int(search_string)
    while(mod_date not in dir_list):
      int_mod_date = int(mod_date)
      int_mod_date+=1
      mod_date = str(int_mod_date)
    print(mod_date)
    converted_date=str(mod_date)
    json_file_name = converted_date + ".json"
    print(json_file_name)
    return json_file_name 



this function iterates through each row of the csv file and appends the correct drought catagory 

In [ ]:
import pandas as pd

Min_date = min_max_date("min",directory)
Max_date = min_max_date("max",directory)
dir_list = folder_toset(directory_path)
df = pd.read_csv(csv_path)

# Use the apply method to apply the function to each row in the DataFrame
df['drought catagory'] = df.apply(lambda row: catchecker(row['lon'],row['lat'],date_match(dir_list,date_change(row['time']),Min_date,Max_date)), axis=1)
  #date is changed from 2022-01-04 to 2022-01-04     
# Save the updated DataFrame to a new CSV file
df.to_csv(csv_out, index=False)